In [ ]:
import os,sys
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import pylab as P

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm, matplotlib.font_manager as fm
sns.set(style="darkgrid")
import matplotlib.patheffects as PathEffects
from matplotlib.ticker import FuncFormatter

%pylab inline

In [ ]:
#nets = pd.read_csv(os.path.join(drop,'Data/nets2015_long.csv'))

In [ ]:
from pandas.api.types import CategoricalDtype
from matplotlib.backends.backend_pdf import PdfPages

In [ ]:
#from shapely.geometry import Point
#from fiona.crs import from_epsg
#import geopandas as gpd
#from pyproj import Proj
#from geopandas.tools import sjoin
#from shapely.geometry import Point
#from fiona.crs import from_epsg
from textwrap import wrap

In [ ]:
#import shapely
#from platform import python_version

#print(python_version())

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.ticker as ticker
import matplotlib.colors as colors
import matplotlib.patheffects as PathEffects

def comma_format(x, p):
    return '$'+format(x, "6,.0f").replace(",", ".")
def percent_format(x, p):
    return format(x, "6,.02f").replace(",", ".")
percent_format = lambda x,pos: '{:,.0f}%'.format(x)
comma_format = lambda x,pos: '{:,.0f}'.format(x)


In [ ]:
# Get current size
fig_size = plt.rcParams["figure.figsize"]
print("Current size:%s" %fig_size)
 
# Set figure width to 12 and height to 9
fig_size[0] = 11
fig_size[1] = 8.5
plt.rcParams["figure.figsize"] = fig_size
print("Current size:%s" %fig_size)
plt.style.use('ggplot')

In [ ]:
pct=lambda x: x/x.sum()

In [ ]:
def easylabeler(breaks,currency=True,prefix='AGE'):
    """
    turn list of breaks into list of strings for binning / cutting.
    Parameters
    ----------
    breaks : list-like
    
    Returns
    -------
    out : list of named strings
    """
    
    if currency:
        ptrn_main="${fr:,.0f} - ${to:,.0f}"
        ptrn_first='Less than ${dt:,.0f}'
        ptrn_last='Greater than ${dt:,.0f}'
    else:
        ptrn_main="{fr:,.0f} - {to:,.0f}"
        ptrn_first='Less than {dt:,.0f}'
        ptrn_last='Greater than {dt:,.0f}'
    
    difflabels=[]
    for f in range(len(breaks)-1):
        
        difflabels.append(ptrn_main.format(fr=breaks[f],to=breaks[f+1]-1))
    ## fix first, last boundary entries
    difflabels[0]=ptrn_first.format(dt=breaks[1])
    difflabels[-1]=ptrn_last.format(dt=breaks[-2])
    return difflabels

### Load / define a few mapping files 

#### Get SOC, NAICS mappings

In [ ]:
BOX_DIR = '/Users/aolsen/Box'
if (os.environ["USERNAME"])=="lzorn":
    BOX_DIR = 'C:\\Users\\lzorn\\Box'

INPUT_PATH  = os.path.join(BOX_DIR, 'Horizon and Plan Bay Area 2050/Blueprint/Final Blueprint Modeling/Protesting Polar Bears/NETS analysis/inputs')
OUTPUT_PATH = os.path.join(BOX_DIR, 'Horizon and Plan Bay Area 2050/Blueprint/Final Blueprint Modeling/Protesting Polar Bears/NETS analysis')
NETS_PATH   = os.path.join(BOX_DIR, 'DataViz Projects/Data Analysis and Visualization/National Establishment Time Series')
THIS_SCRIPT = 'https://github.com/BayAreaMetro/petrale/blob/master/applications/travel_model_lu_inputs/2015/Employment/NETS_2015_work_from_home_analysis.ipynb'

In [ ]:
soc = pd.read_excel(os.path.join(INPUT_PATH,'soc_structure_2010.xls'),skiprows=11,
                   names=[u'Major Group', u'Minor Group', u'Broad Group', u'Detailed Occupation', u'Description'])

soc['maj']=soc['Major Group'].fillna('').str.split('-').apply(lambda x: x[0])

def classifier(df):
    """
    we need to classify each row with the appropriate hierarchy level.
    for each row, we want to get the rightmost value available--
    that represents the finest grained detail class
    """
    
    x = df.iloc[:4].tolist()
    try:
        out = next(s for s in x if not s is np.NaN)
    except:
        out = None
    return  out

def classlevel(s):
    
    try:
        if s[3:]=='0000':
            return 'major'
        elif np.float64(s[3:]) % 100==0:
            return 'minor'
        elif np.float64(s[3:]) % 10==0:
            return 'broad'
        else:
            return 'detail'
    except:
        return 'none'

soc['class']=soc.iloc[:,:4].apply(classifier,axis=1)
soc['hierarchy']=soc['class'].fillna('-1').map(classlevel)

soc[['class','hierarchy','Description']].iloc[1200:1202]

soc[['class','hierarchy','Description']].groupby('hierarchy').size()
soc['Description']=soc.Description.fillna('0').str.lower()
#soc.index=SOC['SOCP_2']

soc['SOCP_2']=soc['class'].apply(lambda x: str(x)[:2])
majorclass = soc.loc[soc['Major Group'].notnull(),['SOCP_2','Description']].copy()
majorclass.index=majorclass.SOCP_2
majorclass=majorclass.Description.to_dict()

soc=soc[soc['class'].notnull()]
soc['class']=soc['class'].str.replace('-','')
#soc[soc.hierarchy=='major'].to_csv(os.path.join(drop,'Documents/Data/_BLS/SOC/soc_structure_2010_major.csv'))

soc_map=soc[soc.hierarchy=='detail'].set_index('Detailed Occupation').Description

soc

In [ ]:
## get abag-naics mapping
names=['NAICS','ABAGSector','mtc11','mtc6','EDDSector','acsnames','acscensus']
naics_mappings = pd.read_excel(os.path.join(INPUT_PATH, 'NAICS_to_ABAG_SECTORS.xlsx'),sheet_name='both')
naics_mappings.columns=names
naics_mappings['NAICS']=naics_mappings['NAICS'].astype(str)

#naics_abag = pd.read_csv(os.path.join(drop, r'Documents\Data\BayArea\Projections 2013\NAICS_to_ABAG_SECTORS.csv'),sep='\t',dtype=object)

naics_abag = naics_mappings.set_index('NAICS').mtc11.to_dict()
naics_mtc = naics_mappings.set_index('NAICS').mtc6.to_dict()
naics_abag_11 = naics_mappings.set_index('NAICS')['mtc11'].to_dict()
naics_mappings.mtc11=naics_mappings.mtc11.str.strip()
naics_mappings.mtc6=naics_mappings.mtc6.str.strip()
naics_mappings.groupby(['acsnames','mtc11']).size().reset_index()
naics_mappings

In [ ]:
# taz = gpd.read_file(os.path.join(drop,'Documents/Data/GIS/zones1454.shp'),crs=from_epsg(3740)).to_crs(from_epsg(3740))
# #taz['county']=taz.county_mtc.map(countymap)
# taz=taz[taz.geometry.notnull()]

# taz['geometry']=taz.buffer(.001)

In [ ]:
# ## read naics-3 values from file
# naicsmap3 =  pd.read_csv(os.path.join(drop,'Documents/Data/Classification/naics_3.csv'),sep='\t',
#   dtype={'Naics_3': object, 'description': object}).set_index('Naics_3').description.to_dict()

In [ ]:
## naics-2 dict
naicsmap = {'11':'Agriculture, Forestry, Fishing and Hunting',
    '21': 'Mining, Quarrying, and Oil and Gas Extraction',
 '22': 'Utilities',
 '23': 'Construction',
 '31': 'Manufacturing',
 '32': 'Manufacturing',
 '33': 'Manufacturing',
 '42': 'Wholesale Trade',
 '44': 'Retail Trade',
 '45': 'Retail Trade',
 '48': 'Transportation and Warehousing',
 '49': 'Transportation and Warehousing',
 '51': 'Information',
 '52': 'Finance and Insurance',
 '53': 'Real Estate and Rental and Leasing',
 '54': 'Professional, Scientific, and Technical Services',
 '55': 'Management of Companies and Enterprises',
 '56': 'Administrative and Support and Waste Management and Remediation Services',
 '61': 'Educational Services',
 '62': 'Health Care and Social Assistance',
 '71': 'Arts, Entertainment, and Recreation',
 '72': 'Accommodation and Food Services',
 '81': 'Other Services except Public Administration',
 '92': 'Public Administration',
 '99': 'Unclassified Establishments'
}

In [ ]:
# ## define dicts relating the field name in the data to the year it represents (e.g. NAICS01: 2001)
# ## we use later to extract the year

# fipstypemap = dict([('fips{:02d}'.format(int(str(x)[2:])),np.dtype('a5')) for x in range(1990,2016)]+\
#               [('naics{:02d}'.format(int(str(x)[2:])),np.dtype('a6')) for x in range(1990,2016)]+\
#               [('sales{:02d}'.format(int(str(x)[2:])),str) for x in range(1990,2016)]+\
#               [('emp{:02d}'.format(int(str(x)[2:])),np.int32) for x in range(1990,2016)])

# fipsmap = dict([('fips{:02d}'.format(int(str(x)[2:])),x) for x in range(1990,2016)]+\
#               [('sales{:02d}'.format(int(str(x)[2:])),x) for x in range(1990,2016)]+\
#               [('naics{:02d}'.format(int(str(x)[2:])),x) for x in range(1990,2016)]+\
#               [('emp{:02d}'.format(int(str(x)[2:])),x) for x in range(1990,2016)])
# fipstypemap['dunsnumber']=str

### Load naics to occupation mappings

In [ ]:
## load naics-to-soc crosswalk for naics 92 public adm data from PUMS 1-year data

naics92_soc_pct = pd.read_csv(os.path.join(INPUT_PATH,'pums_2016_naics92_soc_share.csv'),dtype={'NAICSP_2':str}).rename(columns={'NAICSP_2':'naics_2','soc_x':'occ_code'}).set_index(['naics_2','occ_code']).Total
naics92_soc_pct

In [ ]:
## define 'overflow' industries at 2 digit level

naics_exp={'31': '31-33',
 '32': '31-33',
 '33': '31-33',
 '44': u'44-45',
 '45': u'44-45',
 '48': u'48-49',
 '49': u'48-49'}

In [ ]:
## bay area counties
bayareafips_full = {'06001':'Alameda', '06013':'Contra Costa', '06041':'Marin', '06055':'Napa', '06075':'San Francisco', '06081':'San Mateo', '06085':'Santa Clara', '06097':'Sonoma', '06095':'Solano'}

bayareamsas={'06001': u'San Francisco-Oakland-Hayward, CA',
 '06013': u'San Francisco-Oakland-Hayward, CA',
 '06041': u'San Francisco-Oakland-Hayward, CA',
 '06055': u'Napa, CA',
 #'06069': u'San Jose-Sunnyvale-Santa Clara, CA',
 '06075': u'San Francisco-Oakland-Hayward, CA',
 #'06077': u'Stockton-Lodi, CA',
 '06081': u'San Francisco-Oakland-Hayward, CA',
 '06085': u'San Jose-Sunnyvale-Santa Clara, CA',
 '06087': u'Santa Cruz-Watsonville, CA',
 '06095': u'Vallejo-Fairfield, CA',
 '06097': u'Santa Rosa, CA'}

In [ ]:
## load naics sector to occupation crosswalk from OES research estimates, subsetted to CA only
naics_to_occ=pd.read_excel(os.path.join(INPUT_PATH,'oes_research_data_2019_naics_to_occ_share_ca.xlsx'),dtype={'naics':str,'occ_code':str})
naics_to_occ=naics_to_occ.set_index(['naics','occ_code']).tot_emp
naics_to_occ.index=naics_to_occ.index.set_names('naics_2',level=0)
naics_to_occ.head(2)

In [ ]:
naics_to_occ_w_naics_92 = naics_to_occ.append(naics92_soc_pct)#.reset_index(name='tot_emp')
naics_to_occ_w_naics_92.name='tot_emp'

naics_to_occ_w_naics_92 = naics_to_occ_w_naics_92.reset_index()

naics_to_occ_w_naics_92

In [ ]:
# ## load Dingel's (2020) occupational propensities
# workfromhomeocc_onet_bls=pd.read_csv('https://raw.githubusercontent.com/jdingel/DingelNeiman-workathome/master/occ_onet_scores/output/occupations_workathome.csv')
# workfromhomeocc_onet_bls['OCC_CODE']=workfromhomeocc_onet_bls.onetsoccode.str.split('\.').apply(lambda x: x[0])
# workfromhomeocc_onet_bls=workfromhomeocc_onet_bls.groupby(['OCC_CODE']).teleworkable.mean()


In [ ]:
## load Dingel's (2020) occupational propensities, from O*NET scoring, and mapped to OES categories

# 'https://raw.githubusercontent.com/jdingel/DingelNeiman-workathome/master/onet_to_BLS_crosswalk/output/onet_teleworkable_blscodes.csv')
#workfromhomeocc_onet=pd.read_csv(os.path.join(INPUT_PATH,'onet_teleworkable_blscodes.csv'),sep='\t')
workfromhomeocc_onet=pd.read_csv('https://raw.githubusercontent.com/jdingel/DingelNeiman-workathome/master/onet_to_BLS_crosswalk/output/onet_teleworkable_blscodes.csv')
workfromhomeocc=workfromhomeocc_onet.set_index('OCC_CODE').teleworkable

### Load the employment data
Get from postgres whenever there are query changes. Otherwise, grab from csv made each time postgres is queried.

In [ ]:
float_int = lambda x: pd.to_numeric('{:.0f}'.format(pd.to_numeric(x)),errors='coerce')

In [ ]:
%%time
## large file - get just a few cols

nets = pd.read_csv(os.path.join(NETS_PATH,'nets2015wide.csv'),
                   usecols=['dunsnumber','firstyear','lastyear','fips15','naics15','emp15'],
                   na_values=[''],
                   engine='python')

nets = nets.set_index(['dunsnumber','firstyear','lastyear'])

print("nets length:{} rows size: {} MB".format(len(nets), nets.memory_usage(index=True).sum()*1e-6))
nets

In [ ]:
diffbreaks_5 =[0,25,50,100,250,500,1000,np.inf]
difflabels_5 = easylabeler(breaks=diffbreaks_5,currency=False)
difflabels_5

In [ ]:
# nets_2015=nets.filter(regex='15') # lmz commented out since this appears to do nothing
nets_2015=nets.loc[nets.fips15.notnull()].copy()
nets_2015['naics_2']=nets_2015.naics15.astype(int).apply(lambda x: '{:0<6}'.format(x)).str.slice(0,2)
nets_2015['naics_2_desc']=nets_2015.naics_2.map(naicsmap)
nets_2015['naics_abag']=nets_2015.naics_2.map(naics_abag)
nets_2015['naics_mtc']=nets_2015.naics_2.map(naics_mtc)
nets_2015['naics_2']=nets_2015['naics_2'].replace(naics_exp)
nets_2015['emp_size_cat']=pd.cut(nets_2015.emp15,bins=diffbreaks_5,labels=difflabels_5)
nets_2015['emp_bucket']=pd.cut(nets_2015.emp15,bins=[0,25,np.inf],labels=['0-25 employees','25+ employees'])
nets_2015['STCOUNTY']=nets_2015.fips15.astype(int).apply(lambda x: '{:0>5}'.format(x))
nets_2015['CBSA']=nets_2015.STCOUNTY.map(bayareamsas)
nets_2015=nets_2015[nets_2015.STCOUNTY.isin(bayareafips_full)]

print("nets_2015 length: {} rows; size: {} MB".format(len(nets_2015), nets_2015.memory_usage(index=True).sum()*1e-6))
nets_2015

In [ ]:
## basic summary by indus
nets_2015_by_indus_size = nets_2015.groupby(['CBSA','STCOUNTY','naics_2','naics_2_desc','naics_abag','naics_mtc','emp_bucket']).emp15.sum()
nets_2015_by_indus_size = nets_2015_by_indus_size.reset_index()

print("nets_2015_by_indus_size has {} rows with emp15=null, {} rows with emp15 not null".format(
    pd.isnull(nets_2015_by_indus_size.emp15).sum(),
    pd.notnull(nets_2015_by_indus_size.emp15).sum()))

# select only those few rows with emp15 not null
nets_2015_by_indus_size = nets_2015_by_indus_size.loc[pd.notnull(nets_2015_by_indus_size.emp15),]

print("nets_2015_by_indus_size total emp15: {}".format(nets_2015_by_indus_size.emp15.sum()))

nets_2015_by_indus_size

In [ ]:
## merge emp data with naics-to-occ crosswalk

nets_2015_occ_exp=nets_2015_by_indus_size.merge(naics_to_occ_w_naics_92,on=['naics_2'])
nets_2015_occ_exp['reg']='Bay Area'

## weigh employment by telecommute propensity from Dingel (2020)
nets_2015_occ_exp['emp15_occ']=nets_2015_occ_exp.emp15*nets_2015_occ_exp.tot_emp

print("nets_2015_occ_exp length: {} rows  size: {} MB".format(len(nets_2015_occ_exp), sys.getsizeof(nets_2015_occ_exp)*1e-6))
nets_2015_occ_exp

In [ ]:
## codes in NETS data but *not* in the Dingel work from home matrix

nets_occ_unmatched=list(set(nets_2015_occ_exp.occ_code)-set(workfromhomeocc.index))
#oes_occ_unmatched=list(set(naics_to_occ_w_naics_92.index.get_level_values(1).unique())-set(workfromhomeocc.index))
#onet_occ_unmatched=list(set(workfromhomeocc.index)-set(naics_to_occ_w_naics_92.index.get_level_values(1).unique()))

# nets_occ_unmatched

In [ ]:
soc_det=soc.loc[soc['Detailed Occupation'].notnull()]
soc_det_missing_from_onet_WFH=soc_det[soc_det['Detailed Occupation'].isin(nets_occ_unmatched)].set_index('Detailed Occupation').Description.index

## take the codes that do *not* have a wfh flag and assign the most common flag 
## for occupations in the containing major group. Many of them are "other xxx"
onet_missing_group_imputed=pd.Series(data=soc_det_missing_from_onet_WFH.str.slice(0,2).map(workfromhomeocc.groupby(lambda x: x[:2]).median()),
          index=soc_det_missing_from_onet_WFH)


In [ ]:
## ADD work from home share
nets_2015_occ_exp['wfh_flag']=nets_2015_occ_exp.occ_code.map(workfromhomeocc.append(onet_missing_group_imputed))
nets_2015_occ_exp['emp15_wfh']=nets_2015_occ_exp.emp15_occ*nets_2015_occ_exp.wfh_flag

### Quick summaries

In [ ]:
nets_2015_occ_exp.groupby(['emp_bucket'])['emp15_occ','emp15_wfh'].sum().plot(kind='barh',figsize=[8,5])
title('Bay Area employment, total and work from home potential\nSources: Work from home potential from Dingel (2020)\nIndustry - Occupation matrix from BLS OES 2019 Research Estimates (CA subset)\nEmployment data from National Establishment Timeseries (NETS) 2015')

In [ ]:
fig=plt.figure(figsize=[8,6])
ax=sns.heatmap((nets_2015_occ_exp.groupby(['CBSA','naics_mtc','emp_bucket'])['emp15_wfh'].sum()/\
nets_2015_occ_exp.groupby(['CBSA','naics_mtc','emp_bucket'])['emp15_occ'].sum()).unstack([0]).loc(0)[:,'25+ employees'].reset_index(1,drop=True).T
,
annot=True,fmt=',.2f',linewidths=.5,cmap=cm.coolwarm,
           annot_kws={'fontsize':12})
title('Employment susceptible to telecommuting, using NETS data\nclassified using Dingel (2020) after mapping industry to occupation data\nBay Area CBSAs shown')
plt.tight_layout()
plt.yticks(rotation=0,size=12)
plt.xticks(rotation=45,size=12)


In [ ]:
fig=plt.figure(figsize=[8,6])
ax=sns.heatmap((nets_2015_occ_exp.groupby(['STCOUNTY','naics_mtc','emp_bucket'])['emp15_wfh'].sum()/\
nets_2015_occ_exp.groupby(['STCOUNTY','naics_mtc','emp_bucket'])['emp15_occ'].sum()).unstack([0]).loc(0)[:,'25+ employees'].reset_index(1,drop=True).T
,
annot=True,fmt=',.2f',linewidths=.5,cmap=cm.coolwarm,
           annot_kws={'fontsize':12})
title('Employment susceptible to telecommuting, using NETS data\nclassified using Dingel (2020) after mapping industry to occupation data\nBay Area CBSAs shown')
plt.tight_layout()
plt.yticks(rotation=0,size=12)
plt.xticks(rotation=45,size=12)
#savefig(os.path.join(box, 'RHNA/Analyses/equity/divergence_opportunity_corr.pdf'))

In [ ]:
fig=plt.figure(figsize=[8,6])
ax=sns.heatmap((nets_2015_occ_exp.groupby(['naics_mtc','emp_bucket'])['emp15_wfh'].sum()/\
nets_2015_occ_exp.groupby(['naics_mtc','emp_bucket'])['emp15_occ'].sum()).unstack(),
annot=True,fmt=',.2f',linewidths=.5,cmap=cm.coolwarm,
           annot_kws={'fontsize':12})
title('Employment susceptible to telecommuting, using NETS data\nclassified using Dingel (2020) after mapping industry to occupation data')
plt.tight_layout()
plt.yticks(rotation=0,size=12)
plt.xticks(rotation=45,size=12)
#savefig(os.path.join(box, 'RHNA/Analyses/equity/divergence_opportunity_corr.pdf'))

### Write out spreadsheet with WFH potential share based on industry / occupation and establishment size alone

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format
xl4=pd.ExcelWriter(os.path.join(OUTPUT_PATH,'WFH_By_Sector_V4.xlsx'))

header = pd.DataFrame(data=["Source: {}".format(THIS_SCRIPT),"Table"], columns=["col1"])

## WORK FROM HOME POTENTIAL SHARE, by ESTAB SIZE
header.loc[1] = "Table 1: Work from home potential, by establishment size"
header.to_excel(xl4, 'wfhshare_by_estabsize',index=False,merge_cells=False,header=False)

(nets_2015_occ_exp.groupby(['emp_bucket'])['emp15_wfh'].sum()/\
nets_2015_occ_exp.groupby(['emp_bucket'])['emp15_occ'].sum()).reset_index(name='value').to_excel(xl4,'wfhshare_by_estabsize',index=False,merge_cells=False,startrow=3)

## WORK FROM HOME POTENTIAL SHARE, by ESTAB SIZE, SECTOR
header.loc[1] = "Table 2: Work from home potential, by establishment size and sector"
header.to_excel(xl4, 'wfhshare_by_estabsize_sector',index=False,merge_cells=False,header=False)

(nets_2015_occ_exp.groupby(['naics_mtc','emp_bucket'])['emp15_wfh'].sum()/\
nets_2015_occ_exp.groupby(['naics_mtc','emp_bucket'])['emp15_occ'].sum()).unstack(1).to_excel(xl4,'wfhshare_by_estabsize_sector',index=True,merge_cells=False,startrow=3)

## Work from home potential, by establishment size, sector and MSA
header.loc[1] = "Table 3: Work from home potential, by establishment size, sector and MSA"
header.to_excel(xl4, 'wfhshare_by_cbsa_sector',index=False,merge_cells=False,header=False)

(nets_2015_occ_exp.groupby(['CBSA','naics_mtc'])['emp15_wfh'].sum()/\
nets_2015_occ_exp.groupby(['CBSA','naics_mtc'])['emp15_occ'].sum()).reset_index(name='share').to_excel(xl4,'wfhshare_by_cbsa_sector',index=False,merge_cells=False,startrow=3)

## Work from home potential, by sector
header.loc[1] = "Table 4: Work from home potential, by sector"
header.to_excel(xl4, 'wfhshare_by_sector',index=False,merge_cells=False,header=False)

(nets_2015_occ_exp.groupby(['naics_mtc'])['emp15_wfh'].sum()/\
nets_2015_occ_exp.groupby(['naics_mtc'])['emp15_occ'].sum()).reset_index(name='share').to_excel(xl4,'wfhshare_by_sector',index=False,merge_cells=False,startrow=3)

## Table 5 Work from home potential, by establishment size, sector and county fips
header.loc[1] = "Table 5: Work from home potential, by establishment size, sector and county fips"
header.to_excel(xl4, 'wfhshare_by_county_sector',index=False,merge_cells=False,header=False)

(nets_2015_occ_exp.groupby(['STCOUNTY','naics_mtc'])['emp15_wfh'].sum()/\
nets_2015_occ_exp.groupby(['STCOUNTY','naics_mtc'])['emp15_occ'].sum()).reset_index(name='share').to_excel(xl4,'wfhshare_by_county_sector',index=False,merge_cells=False,startrow=3)

## Table 6 Employees by firm size
header.loc[1] = "Table 6: Employment by Establishment Size"
header.to_excel(xl4, 'emp_by_size',index=False,merge_cells=False,header=False)

emp_by_size = nets_2015.groupby(['emp_bucket'])['emp15'].sum().reset_index()
emp_by_size["emp15_share"] = emp_by_size.emp15/emp_by_size.emp15.sum()
emp_by_size.to_excel(xl4, 'emp_by_size', index=False,merge_cells=False,startrow=3)

xl4.close()